<a href="https://colab.research.google.com/github/pacasarini/web_mining_crawler_scraping/blob/main/projeto_web_mining_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Faculdade Impacta
Projeto de Web Mining & Crawler Scraping
Prof. Fernando de Sequeira

Grupo
Arthur Bassetti
Luis Thiago Amorim de Moura
Paula Cristina Casarini

Para este projeto escolhemos o banco de dados do eBit: https://www.ebit.com.br/reputacao-de-lojas. <br>
Nossa finalidade será realizar uma análise de sentimentos de opiniões sobre empresas por meio das avaliações contidas neste banco de dados.





In [1]:
import requests
response = requests.get('https://www.ebit.com.br/reputacao-de-lojas')
html = response.text
html

'<!doctype html><html lang="pt-br"><head><meta charset="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1"/><script async src="https://www.googletagmanager.com/gtag/js?id=UA-142750785-1"></script><script>function gtag(){dataLayer.push(arguments)}window.dataLayer=window.dataLayer||[],gtag("js",new Date),gtag("config","UA-142750785-1")</script><script>!function(e,t,a,n,g){e[n]=e[n]||[],e[n].push({"gtm.start":(new Date).getTime(),event:"gtm.js"});var m=t.getElementsByTagName(a)[0],r=t.createElement(a);r.async=!0,r.src="https://www.googletagmanager.com/gtm.js?id=GTM-T5ZD8Q7",m.parentNode.insertBefore(r,m)}(window,document,"script","dataLayer")</script><link href="https://nielseniq.com/wp-content/uploads/sites/4/2020/10/cropped-NielsenIQ_Favicon_Green-1.png?w=180" rel="shortcut icon" type="image/x-icon"/><link href="https://nielseniq.com/wp-content/uploads/sites/4/2020/10/cropped-NielsenIQ_Favicon_Green-1.png?w=180" rel="icon" type="image/x-icon"/><link rel="styleshe

In [4]:
from bs4 import BeautifulSoup

bs = BeautifulSoup(html, 'html.parser')
bs

<!DOCTYPE html>
<html lang="pt-br"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-142750785-1"></script><script>function gtag(){dataLayer.push(arguments)}window.dataLayer=window.dataLayer||[],gtag("js",new Date),gtag("config","UA-142750785-1")</script><script>!function(e,t,a,n,g){e[n]=e[n]||[],e[n].push({"gtm.start":(new Date).getTime(),event:"gtm.js"});var m=t.getElementsByTagName(a)[0],r=t.createElement(a);r.async=!0,r.src="https://www.googletagmanager.com/gtm.js?id=GTM-T5ZD8Q7",m.parentNode.insertBefore(r,m)}(window,document,"script","dataLayer")</script><link href="https://nielseniq.com/wp-content/uploads/sites/4/2020/10/cropped-NielsenIQ_Favicon_Green-1.png?w=180" rel="shortcut icon" type="image/x-icon"/><link href="https://nielseniq.com/wp-content/uploads/sites/4/2020/10/cropped-NielsenIQ_Favicon_Green-1.png?w=180" rel="icon" type="image/x-icon"/><link crossorigi